## code for plotting all past years of OCG beach profile 
Using .csv file to complete this 

In [15]:
import pandas as pd 
import numpy as np  
import csv
import matplotlib.pyplot as plt

In [18]:
# import file from computer
OCG2019_profile = pd.read_csv(r"C:\Users\orion\OneDrive\Desktop\OCG_profiles\2019_OCG_profile.csv")
OCG2021_profile = pd.read_csv(r'C:\Users\orion\OneDrive\Desktop\OCG_profiles\2021_OCG_profile.csv')
OCG2022_profile = pd.read_csv(r'C:\Users\orion\OneDrive\Desktop\OCG_profiles\2022_OCG_profile.csv')
OCG2024_profile = pd.read_csv(r'C:\Users\orion\OneDrive\Desktop\OCG_profiles\2024_OCG_profile.csv')

#OCG2019_profile.head() 
OCG2021_profile.head()
#OCG2022_profile.head()
#OCG2024_profile.head()                    

,Date,X,Y
0,2021,0,184
1,2021,5,152
2,2021,10,168
3,2021,15,162
4,2021,20,72


In [20]:
def read_d180_text(csv_file_path):
    '''
    Runs through data files to read files and name headers. To be run alongside read_dH_text and read_dens_text to 
    read all relevant ice core data.
        Inputs:
            csv_file_path (filepath) - desired file path containing d18O data 
    '''
    try:
        df = pd.read_csv(csv_file_path, delimiter='\t', names=['sig_d18O', 'd18O', 'depth_(cm)'])
        debug(f"CSV File Content:\n{df}")
    except Exception as e:
        df = None
        debug(f"Error reading CSV file: {e}")

cores = ['J108', 'J208', 'J409']

d18O_dict = {}
d2H_dict = {}
dens_dict = {}
for core in cores:
    ox_df = read_d180_text(core+'_d18O_lin.txt')
    H_df = read_dH_text(core+'_dH_lin.txt')
    dens_df = read_dens_text(core+'_density.txt')
    
    d18O_dict |= {core:ox_df}
    d2H_dict |= {core:H_df}
    dens_dict |= {core:dens_df}

def dict_to_dataframe_compile(core, d2H_dict, d18O_dict, dens_dict):
    '''
    Runs through dictionaries with individual data streams and merges the data from each core together to produce a 
    dataframe with all three data streams for each ice core.capitalize

        Inputs:
            core (string, default=J108) - name of ice core
            d2H_dict (dictionary) - dictionary containing d2H data from each ice core, core ID is the key, dataframe is
                value
            d18O_dict (dictionary) - dictionary containing d18O data from each ice core, core ID is the key, dataframe is
                value
            dens_dict (dictionary) - dictionary containing density data from each ice core, core ID is the key, dataframe is
                value

    '''

    #Test for validity of core names. Replace with J108 if wrong, and notify user
    valid_cores = ['J108', 'J208', 'J409']

    if type(core) is list:
        core_dict = {}
        for ic in core:
            print('Extracting data from core ', ic, ' as DataFrames.')
            if ic not in valid_cores:
                print('Core selected is not valid. Using J108! Please check your command and try again.')
                ic = 'J108' 
            
            #Looping through the dictionaries to pull out pertinent dataframes
            d2H_df = d2H_dict[ic]
            d18O_df = d18O_dict[ic]
            dens_df = dens_dict[ic]

            #merge these together
            OH_df = d2H_df.merge(d18O_df, how='outer', on='depth_(cm)').dropna()
            merged_single_core_df = OH_df.merge(dens_df, how='outer', on='depth_(cm)').dropna()

            #update the dictionary of cores
            core_dict |= {ic:merged_single_core_df}

            return_var = core_dict
    
    #If single core is entered as an input argument:
    else:
        if type(core) is not str:
            print('Core argument in function must be a string, either J108, J208, or J409!!! Using J108 instead.')
            core = 'J108'
        else:
            d2H_df = d2H_dict[core]
            d18O_df = d18O_dict[core]
            dens_df = dens_dict[core]

            #merge these together
            OH_df = d2H_df.merge(d18O_df, how='outer', on='depth_(cm)').dropna()
            merged_single_core_df = OH_df.merge(dens_df, how='outer', on='depth_(cm)').dropna()

            return_var = merged_single_core_df

    return return_var


KeyError: 1